In [56]:
import bisect
import sortedcontainers  as sc
import itertools as it
from tqdm import tqdm_notebook as tqdm

In [2]:
def iter_fl(fl):
    with open(fl) as infile:
        yield from infile.readlines()

In [42]:
def parse_input(fl_iter):
    seeds = list(map(int, next(fl_iter).split(':')[1].strip().split()))
    range_maps = {
        m: sc.SortedDict()
        for m in [
            "seed2soil",
            "soil2fert",
            "fert2water",
            "water2light",
            "light2temp",
            "temp2hum",
            "hum2loc",
        ]
    }
    lnst2rd = {k[:3]: v for k, v in range_maps.items()}
    for ln in fl_iter:
        if not ln:
            continue
        for st, rd in lnst2rd.items():
            if ln.startswith(st):
                _parse_range(fl_iter, rd)
    return {
        "seeds": seeds,
        "seed_rngs": [(s, s+r-1) for s, r in zip(seeds[::2], seeds[1::2])],
        "range_maps": range_maps,
    }


def _parse_range(fl_iter, rd):
    max_key = 0
    for ln in fl_iter:
        if not ln.strip():
            break
        dst, src, rng = map(int, ln.strip().split())
        max_key = max(max_key, src+rng)
        rd[(src, src+rng-1)] = (dst, dst+rng-1)
    # Fill gaps
    fst_st, fst_end = next(iter(rd))
    if fst_st != 0:
        rd[(0, fst_st-1)] = (0, fst_st-1)
    src_rngs = list(rd)
    for (_, p_end), (c_st, _) in zip(src_rngs, src_rngs[1:]):
        if p_end + 1 == c_st:
            continue
        rd[(p_end+1, c_st-1)] = (p_end+1, c_st-1)
    _, s_end = src_rngs[-1]
    rd[(s_end+1, float('inf'))] = (s_end+1, float('inf'))

In [43]:
test = parse_input(iter_fl("data/day5-test.txt"))
input = parse_input(iter_fl("data/day5-input.txt"))
test

{'seeds': [79, 14, 55, 13],
 'seed_rngs': [(79, 92), (55, 67)],
 'range_maps': {'seed2soil': SortedDict({(0, 49): (0, 49), (50, 97): (52, 99), (98, 99): (50, 51), (100, inf): (100, inf)}),
  'soil2fert': SortedDict({(0, 14): (39, 53), (15, 51): (0, 36), (52, 53): (37, 38), (54, inf): (54, inf)}),
  'fert2water': SortedDict({(0, 6): (42, 48), (7, 10): (57, 60), (11, 52): (0, 41), (53, 60): (49, 56), (61, inf): (61, inf)}),
  'water2light': SortedDict({(0, 17): (0, 17), (18, 24): (88, 94), (25, 94): (18, 87), (95, inf): (95, inf)}),
  'light2temp': SortedDict({(0, 44): (0, 44), (45, 63): (81, 99), (64, 76): (68, 80), (77, 99): (45, 67), (100, inf): (100, inf)}),
  'temp2hum': SortedDict({(0, 68): (1, 69), (69, 69): (0, 0), (70, inf): (70, inf)}),
  'hum2loc': SortedDict({(0, 55): (0, 55), (56, 92): (60, 96), (93, 96): (56, 59), (97, inf): (97, inf)})}}

### Part 1

In [163]:
def part1(seeds, range_maps, debug=False, **kwargs):
    min_loc = float('inf')
    for s in tqdm(seeds, "seeds part 1"):
        loc = _to_loc(s, range_maps, debug=debug)
        if debug:
            print(f"{s=}\t{loc=}")
        min_loc = min(min_loc, loc)
    return min_loc

def _to_loc(seed, range_maps, debug=False):
    x = seed
    if debug:
        print("\n----------------------------------------\n")
        print(f"Seed={x}")
    for k, rm in range_maps.items():
        for st, end in rm:
            if st <= x <= end:
                new_x = rm[(st, end)][0] + x - st
                break
        else:
            new_x = x
        if debug:
            print(f"{k=}\t{x=}\t{new_x=}")
        x = new_x
    return x

In [164]:
print("Part 1 test:", part1(**test))
print("Part 1 input:", part1(**input))

/var/folders/5p/k2zfs2h50pj54h8zssrcmmx00000gn/T/ipykernel_67516/927600657.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(seeds, "seeds part 1"):


seeds part 1:   0%|          | 0/4 [00:00<?, ?it/s]

Part 1 test: 35


seeds part 1:   0%|          | 0/20 [00:00<?, ?it/s]

Part 1 input: 240320250


### Part 2

In [52]:
# size of blind_search
import numpy as np
print(f"10^{round(np.log10(sum(input['seeds'][1::2])))}")

10^9


``` python
pis       pie
  1-2-3-4-5
           pos       poe
            6-7-8-9-10
                  cs       ce
                  9-10-11-12
      cs        ce
      3-4-5-6-7-8
      cs                  ce
      3-4-5-6-7-8-9-10-11-12
```

In [169]:
def find_seed_rngs_rec(cur_ranges, maps, seed_rngs):
    for i, (k, map) in enumerate(maps.items(), start=1):
        new_ranges = []
        # print(f"{i=},Map {k=}\t{map}\tcr={cur_ranges}")
        if not cur_ranges:
            print("Not possible")
            break
        for cur_s, cur_e in cur_ranges:
            for (pis, pie), (pos, poe) in map.items():
                if cur_e < pos or poe < cur_s:
                    # fully out of the range
                    continue
                # some overlap
                overlap_s, overlap_e = max(cur_s, pos), min(cur_e, poe)
                offset = pis-pos
                new_ranges.append((overlap_s+offset, overlap_e+offset))
        # print(f"{new_ranges=}")
        cur_ranges = sorted(new_ranges)
    # print(f"{seed_rngs=}")
    pos_seed_rngs = []
    for (cs, ce), (ss, se) in it.product(cur_ranges, seed_rngs):
        if ce < ss or cs > se:
            continue
        pos_seed_rngs.append((max(cs, ss), min(ce, se)))
    print(f"{pos_seed_rngs=}")
    return pos_seed_rngs


def part2(seed_rngs, range_maps, **kwargs):
    maps = dict(reversed(range_maps.items()))
    loc_rngs = sorted(maps['hum2loc'].values())
    for lr in loc_rngs:
        # print(f"{lr=}")
        # print(f"-----\n")
        rngs: list[tuple[int, int]] = find_seed_rngs_rec(cur_ranges=[lr], maps=maps, seed_rngs=seed_rngs)
        if not rngs:
            print("\n-----------------------------------------\n")
            continue
        seeds = list(it.chain(i for s, e in rngs for i in range(s, e+1)))
        print(f"{len(seeds)=}")
        minloc = part1(
            seeds=seeds,
            range_maps=range_maps,
            debug=False,
        )
        print(f"Part 2: {minloc}")
        break
    return minloc
    

In [170]:
part2(**input)

pos_seed_rngs=[(1256362224, 1256607217), (1684000747, 1689017877), (3521067870, 3527793231), (3527793232, 3556846927), (3569691688, 3585338307)]
len(seeds)=56687803


/var/folders/5p/k2zfs2h50pj54h8zssrcmmx00000gn/T/ipykernel_67516/927600657.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for s in tqdm(seeds, "seeds part 1"):


seeds part 1:   0%|          | 0/56687803 [00:00<?, ?it/s]

Part 2: 28580589


28580589